In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
filepath = "/content/drive/MyDrive/DSCT-FTU/all_stock_data.csv"

In [ ]:
df = pd.read_csv(filepath)
df

,ticker,timestamp,open,high,low,close,volume,bu,sd,fn,fs,fb
0,AAM,2023-01-03 00:00,11284.80,11284.80,11002.68,11190.76,3601.0,2900.0,700.0,0.0,0.0,0.0
1,AAM,2023-01-04 00:00,10673.54,11002.68,10673.54,10908.64,3900.0,1500.0,2400.0,-1145000.0,1145000.0,0.0
2,AAM,2023-01-05 00:00,10438.44,11190.76,10438.44,11190.76,2332.0,2200.0,100.0,0.0,0.0,0.0
3,AAM,2023-01-06 00:00,11002.68,11143.74,10955.66,10955.66,600.0,600.0,0.0,0.0,0.0,0.0
4,AAM,2023-01-09 00:00,11284.80,11284.80,10908.64,10955.66,441.0,400.0,0.0,1200000.0,0.0,1200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1040146,VXT,2025-08-25 00:00,11000.00,11000.00,11000.00,11000.00,0.0,0.0,0.0,0.0,0.0,0.0
1040147,VXT,2025-08-26 00:00,11000.00,11000.00,11000.00,11000.00,0.0,0.0,0.0,0.0,0.0,0.0
1040148,VXT,2025-08-27 00:00,11000.00,11000.00,11000.00,11000.00,0.0,0.0,0.0,0.0,0.0,0.0
1040149,VXT,2025-08-28 00:00,11000.00,11000.00,11000.00,11000.00,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import json

file_path = "/content/drive/MyDrive/DSCT-FTU"

# Đọc file JSON
with open(file_path + "/valid_data.json", "r", encoding="utf-8") as f:
    fi_data = json.load(f)

with open(file_path + "/invalid_tickers.json", "r", encoding="utf-8") as f:
    invalid_tickers = json.load(f)

print("✅ Đã load dữ liệu vào fi_data")
print("Số ticker trong fi_dict:", len(fi_data))
# In thử 1 ticker đầu
first_key = list(fi_data.keys())[0]
print("Ví dụ dữ liệu ticker:", first_key)
print(json.dumps(fi_data[first_key][:2], indent=2))  # in thử 2 phần tử đầu

✅ Đã load dữ liệu vào fi_data
Số ticker trong fi_dict: 1198
Ví dụ dữ liệu ticker: AAM
[
  {
    "organizationId": 2736,
    "ticker": "AAM",
    "year": 2023,
    "quarter": 1,
    "ratios": {
      "ProfitabilityRatio": {
        "EBITMargin": -0.04143011,
        "ROA": 0.072987,
        "ROE": 0.077014,
        "ROIC": 0.03779219
      },
      "ValuationRatios": {
        "BasicEPS": 99.88880655,
        "PriceToBook": 0.52849904,
        "PriceToEarning": 6.9376988
      },
      "Growth": {
        "NetRevenueGrowthYoY": -0.45596839,
        "GrossProfitGrowthYoY": -0.60069024
      }
    }
  },
  {
    "organizationId": 2736,
    "ticker": "AAM",
    "year": 2023,
    "quarter": 2,
    "ratios": {
      "ProfitabilityRatio": {
        "EBITMargin": -0.05010263,
        "ROA": 0.036955,
        "ROE": 0.038859,
        "ROIC": -0.00740753
      },
      "ValuationRatios": {
        "BasicEPS": 69.97369851,
        "PriceToBook": 0.52991354,
        "PriceToEarning": 13.37999074
 

In [ ]:
!pip install --extra-index-url https://fiinquant.github.io/fiinquantx/simple fiinquantx

Looking in indexes: https://pypi.org/simple, https://fiinquant.github.io/fiinquantx/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 5.6 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp312-cp312-linux_x86_64.whl size=567861 sha256=a400df98721636cbb0d98d298cef6c168307a6d6ce951ef24b143df6e815cc11
  Stored in directory: /root/.cache/pip/wheels/ab/d0/26/b82cb0f49ae73e5e6bba4e8462fff2c9851d7bd2ec64f8891e
  Created wheel for msgpack: filename=msgpack-1.0.2-cp312-cp312-linux_x86_64.whl size=15820 sha256=40da065aa415495e74a0d52bf0f6e4e956f0224ce9ae13ac94a3ba79b0f8179d
  Stored in directory: /root/.cache/pip/wheels/67/a6/40/eda0983e595bbf3

In [ ]:
from FiinQuantX import FiinSession, RealTimeData

username = 'DSTC_37@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(
    username=username,
    password=password,
).login()

fi = client.FiinIndicator()


# Tính RSI (mặc định window = 14)
df['rsi'] = fi.rsi(df['close'], window=14)

# Tính MFI (Money Flow Index)
df['mfi'] = fi.mfi(df['high'], df['low'], df['close'], df['volume'], window=14)

# Tính MACD
df['macd'] = fi.macd(df['close'], window_fast=12, window_slow=26)
df['macd_signal'] = fi.macd_signal(df['close'], window_fast=12, window_slow=26, window_sign=9)
df['macd_diff'] = fi.macd_diff(df['close'], window_fast=12, window_slow=26, window_sign=9)

# Tính Bollinger Bands (mặc định window=20, std=2)
df['bollinger_hband'] = fi.bollinger_hband(df['close'], window=20, window_dev=2)
df['bollinger_lband'] = fi.bollinger_lband(df['close'], window=20, window_dev=2)

# Tính ema
df['ema50'] = fi.ema(df['close'], window = 50)
df['ema200'] = fi.ema(df['close'], window = 200)

print(df)


        ticker         timestamp      open      high       low     close  \
0          AAM  2023-01-03 00:00  11284.80  11284.80  11002.68  11190.76   
1          AAM  2023-01-04 00:00  10673.54  11002.68  10673.54  10908.64   
2          AAM  2023-01-05 00:00  10438.44  11190.76  10438.44  11190.76   
3          AAM  2023-01-06 00:00  11002.68  11143.74  10955.66  10955.66   
4          AAM  2023-01-09 00:00  11284.80  11284.80  10908.64  10955.66   
...        ...               ...       ...       ...       ...       ...   
1040146    VXT  2025-08-25 00:00  11000.00  11000.00  11000.00  11000.00   
1040147    VXT  2025-08-26 00:00  11000.00  11000.00  11000.00  11000.00   
1040148    VXT  2025-08-27 00:00  11000.00  11000.00  11000.00  11000.00   
1040149    VXT  2025-08-28 00:00  11000.00  11000.00  11000.00  11000.00   
1040150    VXT  2025-08-29 00:00  11000.00  11000.00  11000.00  11000.00   

         volume      bu      sd         fn  ...         fb        rsi   mfi  \
0       

In [ ]:
tickers = df['ticker'].unique().tolist()
tickers[:10]

['AAM', 'AAT', 'ABR', 'ABS', 'ABT', 'ACB', 'ACC', 'ACG', 'ACL', 'ADP']

In [ ]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

def process_financial_data(df, fi_data, invalid_tickers):
    """
    Xử lý và merge dữ liệu tài chính vào DataFrame giá hằng ngày

    Parameters:
    - df: DataFrame chứa dữ liệu giá hằng ngày
    - fi_data: Dict chứa báo cáo tài chính theo quý
    - invalid_tickers: List các ticker không có báo cáo tài chính

    Returns:
    - DataFrame đã được merge với dữ liệu tài chính
    """

    # Bước 1: Convert fi_data thành DataFrame
    fi_records = []

    for ticker, reports in fi_data.items():
        if ticker in invalid_tickers:
            continue  # Skip invalid tickers

        for report in reports:
            # Tính report_date (ngày cuối quý + 20 ngày)
            year = report['year']
            quarter = report['quarter']

            # Xác định ngày cuối quý
            if quarter == 1:
                quarter_end = datetime(year, 3, 31)
            elif quarter == 2:
                quarter_end = datetime(year, 6, 30)
            elif quarter == 3:
                quarter_end = datetime(year, 9, 30)
            else:  # quarter == 4
                quarter_end = datetime(year, 12, 31)

            # Report date = quarter end + 20 days
            report_date = quarter_end + timedelta(days=20)

            # Extract ratios với safe access
            ratios = report.get('ratios', {})
            profitability = ratios.get('ProfitabilityRatio', {})
            valuation = ratios.get('ValuationRatios', {})
            growth = ratios.get('Growth', {})

            record = {
                'ticker': ticker,
                'year': year,
                'quarter': quarter,
                'report_date': report_date,
                'ROE': profitability.get('ROE', np.nan),
                'EPS': valuation.get('BasicEPS', np.nan),
                'PB': valuation.get('PriceToBook', np.nan),
                'PE': valuation.get('PriceToEarning', np.nan),
                'RevenueGrowth': growth.get('NetRevenueGrowthYoY', np.nan),
                'ProfitGrowth': growth.get('GrossProfitGrowthYoY', np.nan)
            }

            fi_records.append(record)

    # Tạo DataFrame từ financial records
    fi_df = pd.DataFrame(fi_records)

    # Bước 2: Chuẩn bị DataFrame chính
    # Đảm bảo timestamp là datetime
    if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Tạo copy để không thay đổi DataFrame gốc
    result_df = df.copy()

    # Bước 3: Initialize các cột tài chính với NaN
    financial_cols = ['PB', 'PE', 'ROE', 'EPS', 'RevenueGrowth', 'ProfitGrowth']
    for col in financial_cols:
        result_df[col] = np.nan

    # Bước 4: Merge dữ liệu tài chính cho từng ticker
    unique_tickers = result_df['ticker'].unique()

    for ticker in unique_tickers:
        if ticker in invalid_tickers:
            # Ticker không hợp lệ - giữ nguyên NaN
            continue

        # Lấy dữ liệu tài chính của ticker này
        ticker_fi = fi_df[fi_df['ticker'] == ticker].copy()

        if ticker_fi.empty:
            # Không có dữ liệu tài chính cho ticker này
            continue

        # Sort theo report_date
        ticker_fi = ticker_fi.sort_values('report_date')

        # Lấy mask cho ticker này trong df chính
        ticker_mask = result_df['ticker'] == ticker
        ticker_data = result_df[ticker_mask].copy()

        # Merge cho từng ngày
        for idx in ticker_data.index:
            current_date = result_df.loc[idx, 'timestamp']

            # Tìm báo cáo gần nhất có report_date <= current_date
            valid_reports = ticker_fi[ticker_fi['report_date'] <= current_date]

            if not valid_reports.empty:
                # Lấy báo cáo gần nhất
                latest_report = valid_reports.iloc[-1]

                # Cập nhật các cột tài chính
                for col in financial_cols:
                    result_df.loc[idx, col] = latest_report[col]

    return result_df

# Hàm helper để kiểm tra kết quả
def display_sample_results(df, ticker_sample='AAM', n_rows=5):
    """
    Hiển thị mẫu kết quả để kiểm tra
    """
    print(f"Sample results for ticker {ticker_sample}:")
    sample_data = df[df['ticker'] == ticker_sample].head(n_rows)

    # Chỉ hiển thị các cột quan trọng
    display_cols = ['ticker', 'timestamp', 'close', 'PB', 'PE', 'ROE', 'EPS', 'RevenueGrowth', 'ProfitGrowth']
    available_cols = [col for col in display_cols if col in sample_data.columns]

    print(sample_data[available_cols])
    print("\n" + "="*80 + "\n")

# Merge dữ liệu
result_df = process_financial_data(df, fi_data, invalid_tickers)

# Kiểm tra kết quả
display_sample_results(result_df, 'AAM', 10)

Sample results for ticker AAM:
  ticker  timestamp     close  PB  PE  ROE  EPS  RevenueGrowth  ProfitGrowth
0    AAM 2023-01-03  11190.76 NaN NaN  NaN  NaN            NaN           NaN
1    AAM 2023-01-04  10908.64 NaN NaN  NaN  NaN            NaN           NaN
2    AAM 2023-01-05  11190.76 NaN NaN  NaN  NaN            NaN           NaN
3    AAM 2023-01-06  10955.66 NaN NaN  NaN  NaN            NaN           NaN
4    AAM 2023-01-09  10955.66 NaN NaN  NaN  NaN            NaN           NaN
5    AAM 2023-01-10  10814.60 NaN NaN  NaN  NaN            NaN           NaN
6    AAM 2023-01-11  10814.60 NaN NaN  NaN  NaN            NaN           NaN
7    AAM 2023-01-12  10814.60 NaN NaN  NaN  NaN            NaN           NaN
8    AAM 2023-01-13  11096.72 NaN NaN  NaN  NaN            NaN           NaN
9    AAM 2023-01-16  10532.48 NaN NaN  NaN  NaN            NaN           NaN




In [ ]:
result_df.loc[result_df["ticker"] == 'AAM'].tail(35)

,ticker,timestamp,open,high,low,close,volume,bu,sd,fn,...,bollinger_hband,bollinger_lband,ema50,ema200,PB,PE,ROE,EPS,RevenueGrowth,ProfitGrowth
627,AAM,2025-07-14,7690.0,7690.0,7450.0,7450.0,10401.0,500.0,9400.0,0.0,...,7453.736194,6616.263806,7027.377586,7209.422461,0.391492,-25.649747,-0.015257,97.883597,0.573116,10.732740
628,AAM,2025-07-15,7450.0,7470.0,7450.0,7470.0,3936.0,2600.0,1000.0,0.0,...,7516.431259,6614.568741,7044.735327,7212.015272,0.391492,-25.649747,-0.015257,97.883597,0.573116,10.732740
629,AAM,2025-07-16,7470.0,7470.0,7400.0,7400.0,2140.0,100.0,1900.0,0.0,...,7555.484881,6629.515119,7058.667275,7213.885767,0.391492,-25.649747,-0.015257,97.883597,0.573116,10.732740
630,AAM,2025-07-17,7400.0,7400.0,7140.0,7200.0,17298.0,8200.0,8900.0,0.0,...,7565.412858,6644.587142,7064.209735,7213.747600,0.391492,-25.649747,-0.015257,97.883597,0.573116,10.732740
631,AAM,2025-07-18,7200.0,7200.0,7190.0,7200.0,13215.0,8200.0,400.0,21600000.0,...,7573.959199,6661.040801,7069.534844,7213.610808,0.391492,-25.649747,-0.015257,97.883597,0.573116,10.732740
632,AAM,2025-07-21,7200.0,7210.0,7150.0,7150.0,12951.0,6300.0,4800.0,10120000.0,...,7577.561107,6677.438893,7072.690340,7212.977865,0.377170,-276.462599,-0.001372,91.383560,0.891743,57.023466
633,AAM,2025-07-22,7150.0,7200.0,7150.0,7150.0,4602.0,1600.0,800.0,0.0,...,7574.188545,6710.811455,7075.722091,7212.351220,0.377170,-276.462599,-0.001372,91.383560,0.891743,57.023466
634,AAM,2025-07-23,7120.0,7150.0,7050.0,7050.0,18576.0,10300.0,7800.0,0.0,...,7565.483050,6739.516950,7074.713382,7210.735785,0.377170,-276.462599,-0.001372,91.383560,0.891743,57.023466
635,AAM,2025-07-24,7050.0,7380.0,7050.0,7300.0,12986.0,3900.0,7900.0,0.0,...,7579.027642,6755.972358,7083.548151,7211.623986,0.377170,-276.462599,-0.001372,91.383560,0.891743,57.023466
636,AAM,2025-07-25,7300.0,7400.0,7300.0,7380.0,8268.0,3000.0,4500.0,730000.0,...,7600.421490,6772.578510,7095.173714,7213.299369,0.377170,-276.462599,-0.001372,91.383560,0.891743,57.023466


In [ ]:
result_df.to_csv("/content/drive/MyDrive/DSCT-FTU/full_data_1.csv", index=False, encoding="utf-8-sig")